## Importing Packages

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchsummary import summary
import torchvision
from torchvision.models import vit_s_16, ViT_S_16_Weights
from tqdm import tqdm
import os
from torch.optim.lr_scheduler import StepLR
import random
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

ImportError: cannot import name 'vit_s_16' from 'torchvision.models' (/usr/local/lib/python3.10/dist-packages/torchvision/models/__init__.py)

In [ ]:
# Set a random seed for reproducibility
def set_seed(seed_value=42):
    random.seed(seed_value)       # Python random module
    np.random.seed(seed_value)    # Numpy module
    torch.manual_seed(seed_value) # Torch
    os.environ['PYTHONHASHSEED'] = str(seed_value)  # Environment variable

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # if using multi-GPU
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(24)

## Importing the Dataset

In [ ]:
'TODO: Define transformations - crop or resize'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

"""

In order to load the datasets from the shared folder, go to google drive, right click the shared folder, and create a shortcut
to somewhere in your drive.

"""

# Ben's dataset paths
ben_train_dataset_path = "/content/gdrive/MyDrive/24S Classes/Deep Learning/COSC78 Final Project/Train Data"
ben_validation_dataset_path = "/content/gdrive/MyDrive/24S Classes/Deep Learning/COSC78 Final Project/Validation Data"
ben_test_dataset_path = "/content/gdrive/MyDrive/24S Classes/Deep Learning/COSC78 Final Project/Test Data"


# Dawson's dataset paths
daw_train_dataset_path = '/content/gdrive/MyDrive/COSC78 Final Project/Train Data'
daw_validation_dataset_path = '/content/gdrive/MyDrive/COSC78 Final Project/Validation Data'
daw_test_dataset_path = '/content/gdrive/MyDrive/COSC78 Final Project/Test Data'

# Will's dataset paths
will_train_dataset_path = '/content/gdrive/MyDrive/COSC78/COSC78 Final Project/Train Data'
will_validation_dataset_path = '/content/gdrive/MyDrive/COSC78/COSC78 Final Project/Validation Data'
will_test_dataset_path = '/content/gdrive/MyDrive/COSC78/COSC78 Final Project/Test Data'

# Brian's dataset paths
bri_train_dataset_path = '/content/gdrive/MyDrive/Algorithms - Collab/CS 78/COSC78 Final Project/Train Data'
bri_val_dataset_path = '/content/gdrive/MyDrive/Algorithms - Collab/CS 78/COSC78 Final Project/Validation Data'
bri_test_dataset_path = '/content/gdrive/MyDrive/Algorithms - Collab/CS 78/COSC78 Final Project/Test Data'

#% Dataset paths in use %# (currently brian's)
train_dataset_path = bri_train_dataset_path
validation_dataset_path = bri_val_dataset_path
test_dataset_path = bri_test_dataset_path


# Setup datasets using ImageFolder
train_dataset = datasets.ImageFolder(train_dataset_path, transform=transform)
val_dataset = datasets.ImageFolder(validation_dataset_path, transform=transform)
test_dataset = datasets.ImageFolder(test_dataset_path, transform=transform)


# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

## Import and modify the model

In [ ]:
# Load the pre-trained ViT model
pretrained_weights = ViT_S_16_Weights.DEFAULT
vit_model = vit_s_16(weights=pretrained_weights)

# Freeze all the parameters for fine-tuning
for param in vit_model.parameters():
    param.requires_grad = False

# Replace the classifier head
num_classes = 3  # Number of classes in your dataset
vit_model.heads[0] = nn.Linear(vit_model.heads[0].in_features, num_classes)
vit_model.heads[0].requires_grad_ = True

# Print the model summary
#summary(vit_model, input_size=(3, 224, 224))

NameError: name 'ViT_S_16_Weights' is not defined

## Fine tune the model

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(vit_model.parameters())
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Decreases the learning rate by a factor of 0.1 every 10 epochs
torch.manual_seed(24)

In [ ]:
def train_and_validate(model, train_loader, val_loader, optimizer, scheduler, loss_func, epochs=25, patience=5, save_path='best_model.pth'):
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # To store the training and validation loss for plotting or analysis
    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': [], 'train_f1': [], 'val_f1': []}

    best_val_loss = float('inf')
    patience_counter = 0  # Counter for the early stopping

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        train_preds, train_targets = [], []

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

            train_preds.extend(predicted.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        train_accuracy = 100 * correct_train / total_train
        train_f1 = f1_score(train_targets, train_preds, average='weighted')
        epoch_train_loss = train_loss / len(train_loader.dataset)
        history['train_loss'].append(epoch_train_loss)
        history['train_accuracy'].append(train_accuracy)
        history['train_f1'].append(train_f1)

        # Scheduler step (commonly after training step, can be adjusted as per scheduler type)
        scheduler.step()

        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        val_preds, val_targets = [], []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{epochs} - Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = loss_func(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)

                val_preds.extend(predicted.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_accuracy = 100 * correct_val / total_val
        val_f1 = f1_score(val_targets, val_preds, average='weighted')
        epoch_val_loss = val_loss / len(val_loader.dataset)
        history['val_loss'].append(epoch_val_loss)
        history['val_accuracy'].append(val_accuracy)
        history['val_f1'].append(val_f1)

        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Train F1: {train_f1:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%, Validation F1: {val_f1:.4f}')

        # Check for improvement in validation loss
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), save_path)  # Save the best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Stopping early after {epoch + 1} epochs due to no improvement in validation loss.")
                model.load_state_dict(torch.load(save_path))  # Load the best model weights
                break

    return history

In [ ]:
history = train_and_validate(vit_model, train_loader, val_loader, optimizer, scheduler, loss_func, epochs=25)

Epoch 1/25 - Training:   0%|          | 0/12 [12:54<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Get test accuracy and scores
test_loss = 0
correct_test = 0
total_test = 0
test_preds, test_targets = [], []

for images, labels in tqdm(test_loader, desc=f"Test dataset"):
  images, labels = images.to(device), labels.to(device)
  optimizer.zero_grad()
  outputs = mobilenet_model(images)
  loss = loss_func(outputs, labels)

  test_loss += loss.item() * images.size(0)
  _, predicted = torch.max(outputs.data, 1)
  correct_test += (predicted == labels).sum().item()
  total_test += labels.size(0)

  test_preds.extend(predicted.cpu().numpy())
  test_targets.extend(labels.cpu().numpy())

test_accuracy = 100 * correct_test / total_test
test_f1 = f1_score(test_targets, test_preds, average='weighted')
test_loss = test_loss / len(test_loader.dataset)

print(f"Test accuracy: {test_accuracy:.2f}%, Test f1 score: {test_f1:.4f}, Test loss: {test_loss:.4f}")



In [ ]:
# Plotting training and validation accuracy
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.plot(history['train_accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

# Plotting training and validation F1 score
plt.subplot(1, 3, 2)
plt.plot(history['train_f1'], label='Train F1 Score')
plt.plot(history['val_f1'], label='Validation F1 Score')
plt.title('Model F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 3, 3)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()